## **Automatique validation**



#### **1- Connect to your instance with the sdk**

In [28]:
from toucan_data_sdk import ToucanDataSdk
import pandas as pd
import getpass

instance = 'scc'
small_app = 'lucent'
instance_url = f"https://api-{instance}.toucantoco.com"
username = 'toucantoco'
try:
    auth = get_auth(instance)
except Exception:
    auth = (username, getpass.getpass())
    
sdk = ToucanDataSdk(instance_url, small_app=small_app, auth=auth)

In [29]:
import numpy as np
import json

#### **2- Get the datasources and etl**

In [30]:
# You can choose specific ouput domains
data_sources_dict = sdk.get_datasources()
etl = sdk.get_etl()


#### **3-package**

In [38]:
def expected_columns(data_source):
    return [c for c in list(data_source.columns) if not c.startswith('_')]

In [39]:
def expected_data_type(data_source):
    expected = dict(
        data_source.dtypes.apply(lambda x: 'number' if np.issubdtype(x, np.number) else 'string')
    )
    
    # Ce que je fais ci apres est ameilliorable... Des feedbacks ?
    expected_corrected = {}
    for col_name, dtype in expected.items():
        if col_name.startswith('_'):
            continue
        if dtype == 'number':
            expected_corrected[col_name] = dtype
        if dtype == 'string':
            try:
                if not data_source[col_name].astype(str).str.replace('.','').str.replace(',','').str.isdigit().any():
                    expected_corrected[col_name] = dtype
            except Exception:
                pass
    # --
        
    return expected_corrected

In [40]:
expected = {
    'columns': expected_columns,
    'data_type': expected_data_type
}

#### **4- update etl**

In [41]:
if __name__ == 'ipython':
    for datasources_conf in etl['DATA_SOURCES']:
        data_source = data_sources_dict[datasources_conf['domain']]
        validation = []

        for type_ in ['columns', 'data_type']:
            validation_step = {'type': type_}
            validation_step['expected'] = expected[type_](data_source)
            validation += validation_step

In [17]:
requete = sdk.client.config.etl.put(json = etl)
if requete.status_code == 200:
    print('oh yeah! ... go check if validation pass now :)')

oh yeah! ... go check if validation pass now :)
